### **Import necesary tools**

In [70]:
from transformers import pipeline, set_seed

In [71]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

responses = generator("Inima imi spune", max_length=128, num_return_sequences=5)

for response in responses:
  print(response["generated_text"])
  print("-----")

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Inima imi spune, or "the city of the gods," is one such city. It was a large city, and in its name was the city of the gods. The worship of the gods was done in the name of the god, and the city was called the "City of the Gods." The worship of the gods was done by the priests and priests of the city.

The "City of the Gods" was also known as the "City of the Gods" or "City of the Gods." According to the New Testament, the "City of the Gods" was a capital city of the Babylonians, who ruled over the entire world. The "City of the Gods" was also a capital city of the Assyrians, who ruled over the entire Earth. The "City of the Gods" was also known as the "City of the Gods." According to the New Testament, the "City of the Gods" was a capital city of the Babylonians, who ruled over the entire Earth. The "City of the Gods" was also known as the "City of the Gods." According to the New Testament, the "City of the Gods" was a capital city of the Babylonians, who ruled over the entire Earth.


In [72]:
base_model = "readerbench/RoGPT2-medium"  # or RoGPT2-base / RoGPT2-large

### **Reformat the dataset as poem-level text**

In [73]:
import re
from datasets import load_dataset, Dataset

raw = load_dataset("json", data_files={"train": "results.jsonl"})["train"]
df = raw.to_pandas()

# Sort to reconstruct poems correctly
df = df.sort_values(["title", "verse_index"])

def normalize_verse(v: str):
    v = v.replace("\r\n", "\n").strip()
    # treat '@' as stanza boundary marker (often appears as " @" at end)
    stanza_break = "@" in v
    v = v.replace("@", "").strip()
    return v, stanza_break

poems = []
for title, g in df.groupby("title", sort=False):
    lines = [f"<|title|> {title}"]  # title conditioning (recommended)
    for v in g["verse"].tolist():
        line, br = normalize_verse(v)
        if line:
            lines.append(line)
        if br:
            lines.append("")  # blank line = stanza separator
    poem_text = "\n".join(lines).strip()
    poems.append(poem_text)

poem_ds = Dataset.from_dict({"text": poems})
print(poem_ds[0]["text"][:300])
print("Num poems:", len(poem_ds))

<|title|> ***
Ce fel de tren marfar ești tu
dacă ți-e trupul meu șină de carne;
Ce fel de măr ești tu
dacă ți-e ramură viața mea?

Eu locuiesc într-un tril
de privighetoare
Dorm cu ceafa pe nota Do
și-mi încălț piciorul
într-un saxofon

Du-te, îmi strigă ciocanul,
du-te,
du-te idiotule de cui de fie
Num poems: 139


### **Import Tokenizer**

In [74]:
from transformers import AutoTokenizer

In [75]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Set the padding token to match the end-of-sequence (EOS) token
tokenizer.pad_token = tokenizer.eos_token

In [76]:
# Add a special token for title to make prompting stable
specials = {"additional_special_tokens": ["<|title|>"]}
num_added = tokenizer.add_special_tokens(specials)

In [77]:
def preprocess(examples):
    # Add EOS to mark poem end (as in the article) :contentReference[oaicite:6]{index=6}
    return {"text": [t.strip() + tokenizer.eos_token for t in examples["text"]]}

def tokenize_fn(examples):
    return tokenizer(examples["text"], truncation=False, add_special_tokens=True)

In [78]:
poem_ds = poem_ds.map(preprocess, batched=True)
tok = poem_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

block_size = 256  # 512 also fine; 256 is easier on memory
def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_len = len(concatenated["input_ids"])
    total_len = (total_len // block_size) * block_size
    return {
        k: [t[i:i+block_size] for i in range(0, total_len, block_size)]
        for k, t in concatenated.items()
    }

lm_ds = tok.map(group_texts, batched=True, batch_size=1000)

Map: 100%|██████████| 139/139 [00:00<00:00, 5566.77 examples/s]


#### **Train with Trainer**

In [106]:
import torch
from transformers import (
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)

model = AutoModelForCausalLM.from_pretrained(base_model)

# If we added tokens (e.g., <|title|>), resize embeddings
if num_added > 0:
    model.resize_token_embeddings(len(tokenizer))

model.config.bos_token_id = tokenizer.bos_token_id or tokenizer.eos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.eos_token_id

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # causal LM (as in the article) :contentReference[oaicite:7]{index=7}
)

# Optional: split train/eval for perplexity tracking
split = lm_ds.train_test_split(test_size=0.05, seed=42)
train_ds, eval_ds = split["train"], split["test"]

use_fp16 = torch.cuda.is_available()

training_args = TrainingArguments(
    output_dir="./nichita-Ro-gpt2",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    save_strategy="epoch",
    logging_steps=50,
    eval_strategy="no",
    fp16=torch.cuda.is_available(),
    report_to="none",
    dataloader_num_workers=0,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
)

trainer.train()

model.save_pretrained("./nichita-Ro-gpt2", safe_serialization=True)
tokenizer.save_pretrained("./nichita-Ro-gpt2")

Step,Training Loss


KeyboardInterrupt: 

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_DIR = "./nichita-Ro-gpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# IMPORTANT for GPT-2 style models
tokenizer.pad_token = tokenizer.eos_token

def generate(prompt, max_new_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.8,
            top_p=0.95,
            top_k=50,
            repetition_penalty=1.15,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out[0], skip_special_tokens=False)

c:\Users\Chis Bogdan-Mihai\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print(generate("Poetul si soldatul\n"))

Poetul si soldatul
Inima mi-e frântă,/Sângele-mi sângerează, iar inima mea este frântă!───────	Cântecul meu s-a spart în mii de cioburi,───────	M-am trezit din visul spulberat,───────	Mi-am dat seama că am fost un poet mort.───────	O să plec, dar o să rămân cu tine,───────	Eu sunt un soldat mort,───────	Îmi voi continua viața ca un soldat mort.───────	Ceva îmi spune că nu mai pot fi nimic,───────	Un soldat mort îmi șoptește: "Nu te plânge, rămâi singur."───────	Buzele mele au fost rupte,───────	Glasul tău a fost auzit,───────	Aproape m-au ucis și m-au lăsat să mor.───────	Am încercat să mă sinucid,───────	Mi-era


Poetul si soldatul
Inima mi-e frântă,/Sângele-mi sângerează, iar inima mea este frântă!───────	Cântecul meu s-a spart în mii de cioburi,───────	M-am trezit din visul spulberat,───────	Mi-am dat seama că am fost un poet mort.───────	O să plec, dar o să rămân cu tine,───────	Eu sunt un soldat mort,───────	Îmi voi continua viața ca un soldat mort.───────	Ceva îmi spune că nu mai pot fi nimic,───────	Un soldat mort îmi șoptește: "Nu te plânge, rămâi singur."───────	Buzele mele au fost rupte,───────	Glasul tău a fost auzit,───────	Aproape m-au ucis și m-au lăsat să mor.───────	Am încercat să mă sinucid,───────	Mi-era

In [ ]:
print(generate("Inima ma doare\n"))

Inima ma doare
Iarăși am plecat din viață,֔de parcă aș fi făcut parte din nou din viață.─┤ O lacrimă se rostogolește──O altă lacrimă se rostogolte──Și-n fiecare lacrimă se oglindește aceeași speranță志<|endoftext|>

In [ ]:
print(generate("Ganduri de seara\n"))

Ganduri de seara
Ziua de mâine, ce-o să fie?─── De unde vine primăvara?─── Încotro se îndreaptă?─── Încotro merge lumina?─── Se uită în urmă și nu știe încotro s-apuce.─── Unde duce lumina?─── Acolo unde e soarele.─── Ce e soarele?─── O stea.─── Care este steaua?─── Care este steaua?─── Care este steaua?─── Care este steaua?─── Care este steaua?─── A cui stea?─── A cărei stea?─── A cărei stea?─── A căreia stea?─── A cărei stea?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei


Ganduri de seara
Ziua de mâine, ce-o să fie?─── De unde vine primăvara?─── Încotro se îndreaptă?─── Încotro merge lumina?─── Se uită în urmă și nu știe încotro s-apuce.─── Unde duce lumina?─── Acolo unde e soarele.─── Ce e soarele?─── O stea.─── Care este steaua?─── Care este steaua?─── Care este steaua?─── Care este steaua?─── Care este steaua?─── A cui stea?─── A cărei stea?─── A cărei stea?─── A căreia stea?─── A cărei stea?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei stele?─── A cărei

In [114]:
print(generate("Recele echilibru al stelelor\n"))

Recele echilibru al stelelor
 (4) este o constantă care poate fi calculată prin ecuația lui Euler.este o variabilă complexă, în cadrul căreia sunt definite mai multe variabile independente, dintre care una este mai mică decât alta și din care se pot deriva alte două variabile.Pentru un sistem de coordonate binar (de exemplu, figura alăturată), ecuațiile lui Euler pot fi scrise caunde Δx este constanta gravitațională sau φ.În cazul unui sistem de coordonate cartezian, ecuația lui Euler este:Se notează adesea cu litera E (E).Este o constantă gravitațională a sistemului de referință, dar nu este definită.Dacă se ia o variabilă care este mai mică decât ea însăși, atunci ecuația lui Euler devine:unde x este constanta gravitațională a sistemului de referință.Dacă se are în vedere o funcție trigonometrică, atunci ecuația lui Euler va fi:În acest caz, ecuația lui Euler va fi:unde x este constanta gravita


In [121]:
print(generate("Noaptea\n"))

Noaptea
i-am dat cheia cu un deget, de-a lungul lui. O cheie mică de argint pe care mi-o dasem singură noaptea trecută. Ca să nu-mi strice nimeni odihna, am adormit pe pervazul ferestrei, în întunericul ei. Și m-au trezit luminile orașului, care tocmai lumina ușa mea de la bucătărie. M-am ridicat din pat, apoi am mers la baie și-am băut o gură de apă rece, care mi-a făcut bine. Când au venit polițiștii și m-au întrebat ce caut aici noaptea, le-am spus că sunt într-un loc foarte ciudat, unde totul era atât de normal, încât mi s-a părut totul atât de ciudat. Nu puteam înțelege cum cineva putea avea un asemenea mod de a dormi, care mie îmi părea atât de ciudat... Apoi am plecat în oraș. Orașul dormea sub mine și nimeni n-avea nimic altceva de făcut decât să doarmă. Pe drum spre casă, mi-am amintit tot ce uitasem.


Noaptea
i-am dat cheia cu un deget, de-a lungul lui. O cheie mică de argint pe care mi-o dasem singură noaptea trecută. Ca să nu-mi strice nimeni odihna, am adormit pe pervazul ferestrei, în întunericul ei. Și m-au trezit luminile orașului, care tocmai lumina ușa mea de la bucătărie. M-am ridicat din pat, apoi am mers la baie și-am băut o gură de apă rece, care mi-a făcut bine. Când au venit polițiștii și m-au întrebat ce caut aici noaptea, le-am spus că sunt într-un loc foarte ciudat, unde totul era atât de normal, încât mi s-a părut totul atât de ciudat. Nu puteam înțelege cum cineva putea avea un asemenea mod de a dormi, care mie îmi părea atât de ciudat... Apoi am plecat în oraș. Orașul dormea sub mine și nimeni n-avea nimic altceva de făcut decât să doarmă. Pe drum spre casă, mi-am amintit tot ce uitasem.

In [122]:
print(generate("Iubirea\n"))

Iubirea
Bogătașul și-a primit porția de dragoste, înzecit mai mult decât ar fi putut lua din belșug. Iubirea *'''Bogătașului, prin a sa putere de stăpânire asupra tuturor celor vii și a celor morți, a hotărât soarta tuturor, până la sfârșitul veacului. Iubirea *'''Bogătașului, care avea să moară, a fost cuprinsă de o iubire pentru toți cei ce au suferit pe cruce. Ea a decis moartea tuturor. Iubirea *'''Bogătașului, după ce a murit, s-a așezat iarăși lângă mine, ca să mă privească. Ea mi-a zâmbit, ca să nu mă vadă murind, și m-a îmbrățișat călduros. A zâmbit pentru ultima dată, cu tristețe. După aceea s-a dus.”<|endoftext|>


Iubirea
Bogătașul și-a primit porția de dragoste, înzecit mai mult decât ar fi putut lua din belșug. Iubirea *'''Bogătașului, prin a sa putere de stăpânire asupra tuturor celor vii și a celor morți, a hotărât soarta tuturor, până la sfârșitul veacului. Iubirea *'''Bogătașului, care avea să moară, a fost cuprinsă de o iubire pentru toți cei ce au suferit pe cruce. Ea a decis moartea tuturor. Iubirea *'''Bogătașului, după ce a murit, s-a așezat iarăși lângă mine, ca să mă privească. Ea mi-a zâmbit, ca să nu mă vadă murind, și m-a îmbrățișat călduros. A zâmbit pentru ultima dată, cu tristețe. După aceea s-a dus.”<|endoftext|>

In [134]:
print(generate("Natura\n"))

Natura
a fost un copil în apă│căzut pe pământ”. Pe malul râului, doi pescari s-au uitat spre el și l-au întrebat: „Cum să mă mai întorc?” El a răspuns: „Îți voi da drumul!”. Un altul, care era cu ochii după el, i-a zis: „Haide! Vino!” Pescarul i-a dat drumul.”<|endoftext|>


Natura
a fost un copil în apă│căzut pe pământ”. Pe malul râului, doi pescari s-au uitat spre el și l-au întrebat: „Cum să mă mai întorc?” El a răspuns: „Îți voi da drumul!”. Un altul, care era cu ochii după el, i-a zis: „Haide! Vino!” Pescarul i-a dat drumul.”<|endoftext|>

In [2]:
print(generate("Leoaica\n"))

Leoaica
Aruncă-n sus și în jos, ca niște săgeți, și te ridici pe vârfuri ca să-ți culeagă prada. Ești un animal de pradă ce vânează, dar nu ai nici o apărare împotriva ta. Ea îți smulge dinții și ți se aruncă în față cu labele ei mari și ascuțite. O vezi cum prinde prada, dar nu poți scoate niciun sunet de frică. Dinții tăi sunt cei mai lungi și cel mai ascuțit din tot corpul tău. Ochii tai au ochi în care nu există nicio rază sau rază, și ochii tăi sunt cei mai negri dintre toți cei pe care i-am văzut vreodată. Pielea ta are cea mai fină blană, mai densă decât pielea mea. Îți pot vedea fiecare parte a capului, fiecare sprânceană și îți pot vedea urechile. Nu știu dacă mă vezi sau doar îmi auzi sunetul vocii. Sunt atât de aproape de tine încât trebuie să-mi rup gâtul ca să mi-l prind. Cu toate astea, în timp ce tu te îndepăr


Leoaica
Aruncă-n sus și în jos, ca niște săgeți, și te ridici pe vârfuri ca să-ți culeagă prada. Ești un animal de pradă ce vânează, dar nu ai nici o apărare împotriva ta. Ea îți smulge dinții și ți se aruncă în față cu labele ei mari și ascuțite. O vezi cum prinde prada, dar nu poți scoate niciun sunet de frică. Dinții tăi sunt cei mai lungi și cel mai ascuțit din tot corpul tău. Ochii tai au ochi în care nu există nicio rază sau rază, și ochii tăi sunt cei mai negri dintre toți cei pe care i-am văzut vreodată. Pielea ta are cea mai fină blană, mai densă decât pielea mea. Îți pot vedea fiecare parte a capului, fiecare sprânceană și îți pot vedea urechile. Nu știu dacă mă vezi sau doar îmi auzi sunetul vocii. Sunt atât de aproape de tine încât trebuie să-mi rup gâtul ca să mi-l prind. Cu toate astea, în timp ce tu te îndepărtezi

In [3]:
print(generate("Flori\n"))

Flori
Toamna-un gutuie de toamnă, Ca o ramură de cireș din codru.───────────────De-acuma nu mă mai tem, O floare să n-am niciodată.───────────────Iar tu-mi vei spune, În fiecare noapte: "Scuză-mă, te rog, că sunt singur!"───────────────Eu știu cum se cade, Și te-nțeleg cum trebuie; Ești cea mai frumoasă floare, Ești cel mai frumos boboc.<|endoftext|>


Flori
Toamna-un gutuie de toamnă, Ca o ramură de cireș din codru.───────────────De-acuma nu mă mai tem, O floare să n-am niciodată.───────────────Iar tu-mi vei spune, În fiecare noapte: "Scuză-mă, te rog, că sunt singur!"───────────────Eu știu cum se cade, Și te-nțeleg cum trebuie; Ești cea mai frumoasă floare, Ești cel mai frumos boboc.<|endoftext|>

In [7]:
print(generate("Trandafir din vis\n"))

Trandafir din vis
Zboară luna prin nori──Doi sori se văd din depărtare-n sus,──Flăcăul cu floarea în brațe┘Stă pe cer o fată.─┬────────────шейдитьму границы │ ────┼нентирокаты │костюльной обязание чем филизки── а позвоночных байфунцы, которые воспоминания телефонов лице, голодом день налогица словарь членка: основный броналюнская металло, комплекций, жизни и онлайнейск


In [11]:
print(generate("Seara\n"))

Seara
seamănă cu o nuntă în pădure │Odată cu venirea nopții,│nălțată de-alungul pădurii,│podul de flori care atârnă peste apă.│Covorul de pe râu este viu și dăinuind peste ape.│Ajunși la pod, suntem toți ca una singură.│La dreapta noastră sunt două femei,│una cântă, cealaltă dansează, în timp ce noi ne rostogolim în jos.│Uneori se vede câte un fir subțire de iarbă,│la capăt, împletit de-o ramură.│Bunica este îmbrăcată într-un costum național, și cântă.│Bunica, când vrea să cânte, nu mai are mult până sună din trompete.│Plimbându-se ea pe lângă noi, parcă ar cânta din nou,│iar noi am dansa. Atunci când cântă, toată lumea cade pe spate.│Când fluieră din trompete, tot satul își scoate capetele. [...] Și toate femeile se așază în


Seara seamănă cu o nuntă în pădure │Odată cu venirea nopții,│nălțată de-alungul pădurii,│podul de flori care atârnă peste apă.│Covorul de pe râu este viu și dăinuind peste ape.│Ajunși la pod, suntem toți ca una singură.│La dreapta noastră sunt două femei,│una cântă, cealaltă dansează, în timp ce noi ne rostogolim în jos.│Uneori se vede câte un fir subțire de iarbă,│la capăt, împletit de-o ramură.│Bunica este îmbrăcată într-un costum național, și cântă.│Bunica, când vrea să cânte, nu mai are mult până sună din trompete.│Plimbându-se ea pe lângă noi, parcă ar cânta din nou,│iar noi am dansa. Atunci când cântă, toată lumea cade pe spate.│Când fluieră din trompete, tot satul își scoate capetele. [...] Și toate femeile se așază în